In [2]:
!pip install faiss-cpu

import faiss
print(faiss.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.2 MB/s eta 0:00:00:00:0100:01
1.13.1


In [3]:
import numpy as np
import pandas as pd
import faiss
from tqdm import tqdm
import os
import gc

# ============================================================================
# CẤU HÌNH KỸ THUẬT (STRICT MODE)
# ============================================================================
CONFIG = {
    'EMBED_DIR': "/kaggle/input/cafa6-embeds", 
    'TRAIN_TERMS': "/kaggle/input/cafa-6-protein-function-prediction/Train/train_terms.tsv",
    'TRAIN_IDS': "/kaggle/input/cafa6-embeds/train_ids.txt",
    'TEST_IDS': "/kaggle/input/cafa6-embeds/test_ids.txt",
    
    'K_NEIGHBORS': 10,       # K nhỏ
    'HOMOLOGY_THRESHOLD': 0.85, 
    'MIN_TERM_VOTES': 2,
    
    'BATCH_SIZE': 1000,
}

def run_consensus_knn_miner():
    print("🚀 STARTING CONSENSUS HOMOLOGY MINER (FAISS CPU)...")
    
    # 1. Load Data & Embeddings (Giữ nguyên như trước)
    print("   1. Loading Resources...")
    df = pd.read_csv(CONFIG['TRAIN_TERMS'], sep='\t', usecols=['EntryID', 'term'])
    labels_dict = df.groupby('EntryID')['term'].apply(list).to_dict()
    del df; gc.collect()

    with open(CONFIG['TRAIN_IDS']) as f: train_ids = np.array([l.strip() for l in f])
    with open(CONFIG['TEST_IDS']) as f: test_ids = np.array([l.strip() for l in f])
    
    X_train = np.load(os.path.join(CONFIG['EMBED_DIR'], "train_embeds.npy")).astype('float32')
    X_test = np.load(os.path.join(CONFIG['EMBED_DIR'], "test_embeds.npy")).astype('float32')
    
    faiss.normalize_L2(X_train)
    faiss.normalize_L2(X_test)

    # 2. Build Index
    print("   2. Building FAISS Index...")
    index = faiss.IndexFlatIP(X_train.shape[1])
    index.add(X_train)
    del X_train; gc.collect()

    # 3. Search & Filter (LOGIC CẬP NHẬT)
    print(f"   3. Mining with Consensus Check...")
    output_file = "knn_homology_candidates.tsv"
    
    with open(output_file, "w") as f_out:
        for i in tqdm(range(0, X_test.shape[0], CONFIG['BATCH_SIZE'])):
            batch_test = X_test[i : i + CONFIG['BATCH_SIZE']]
            D, I = index.search(batch_test, CONFIG['K_NEIGHBORS'])
            
            batch_lines = []
            
            for j in range(len(batch_test)):
                # Lọc ngay từ đầu: Hàng xóm thứ 2 phải xịn (Sim > 0.85)
                # Nếu không thì chắc chắn không đủ 2 vote
                if D[j, 1] < CONFIG['HOMOLOGY_THRESHOLD']: continue
                
                pid = test_ids[i + j]
                term_scores = {} # Tổng điểm sim
                term_votes = {}  # Đếm số người vote
                
                # Duyệt qua các hàng xóm
                for k in range(CONFIG['K_NEIGHBORS']):
                    sim = float(D[j, k])
                    if sim < CONFIG['HOMOLOGY_THRESHOLD']: break 
                    
                    neighbor_pid = train_ids[I[j, k]]
                    terms = labels_dict.get(neighbor_pid, [])
                    
                    for t in terms:
                        term_scores[t] = term_scores.get(t, 0.0) + sim
                        term_votes[t] = term_votes.get(t, 0) + 1
                
                if not term_scores: continue
                
                # Lọc và Ghi
                valid_terms = []
                for t, raw_sum in term_scores.items():
                    votes = term_votes[t]
                    
                    # [QUAN TRỌNG] Chỉ lấy nhãn có >= 2 người vote
                    if votes < CONFIG['MIN_TERM_VOTES']: continue
                    
                    # Tính điểm trung bình của những người vote
                    # Ví dụ: 2 người vote (0.9, 0.88) -> Avg = 0.89 (Rất cao)
                    avg_score = raw_sum / votes
                    
                    # Chặn dưới lần cuối (để chắc chắn không có rác)
                    if avg_score >= 0.85:
                        valid_terms.append((t, avg_score))
                
                # Sort lấy top (chỉ cần lấy ít thôi vì đây là Homology mạnh)
                valid_terms.sort(key=lambda x: x[1], reverse=True)
                
                for term, score in valid_terms[:50]:
                    batch_lines.append(f"{pid}\t{term}\t{score:.4f}\n")
            
            f_out.write("".join(batch_lines))
            
    print(f"✅ DONE! Consensus Candidates saved to {output_file}")

if __name__ == "__main__":
    run_consensus_knn_miner()

🚀 STARTING CONSENSUS HOMOLOGY MINER (FAISS CPU)...
   1. Loading Resources...
   2. Building FAISS Index...
   3. Mining with Consensus Check...


100%|██████████| 225/225 [07:34<00:00,  2.02s/it]

✅ DONE! Consensus Candidates saved to knn_homology_candidates.tsv


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# CẤU HÌNH
KNN_FILE = "knn_homology_candidates.tsv"
IA_FILE = "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv"

def analyze_knn_candidates():
    print(f"📊 ANALYZING {KNN_FILE}...\n")
    
    # 1. Load Data
    try:
        df = pd.read_csv(KNN_FILE, sep='\t', names=['pid', 'term', 'score'])
    except FileNotFoundError:
        print("❌ Error: File not found!")
        return
    except pd.errors.EmptyDataError:
        print("⚠️ Warning: File is EMPTY! (Threshold quá gắt, không tìm thấy hàng xóm nào).")
        return

    # 2. Load IA
    print("   Loading IA weights...")
    ia_map = {}
    try:
        with open(IA_FILE, 'r') as f:
            for line in f:
                p = line.strip().split('\t')
                if len(p) >= 2: ia_map[p[0]] = float(p[1])
    except: pass
    
    # Map IA vào DataFrame
    df['ia'] = df['term'].map(ia_map).fillna(0)
    
   # =========================================================================
    # 3. BÁO CÁO THỐNG KÊ (CẬP NHẬT: IA THEO KHOẢNG CHI TIẾT)
    # =========================================================================
    
    n_prots = df['pid'].nunique()
    n_terms = df['term'].nunique()
    n_rows = len(df)
    
    print("-" * 40)
    print(f"🔹 TỔNG QUAN:")
    print(f"   - Tổng số dòng (Predictions): {n_rows:,}")
    print(f"   - Số Protein được 'Cứu' (Covered): {n_prots:,}")
    print(f"   - Số Nhãn GO xuất hiện: {n_terms:,}")
    print(f"   - Trung bình số nhãn/protein: {n_rows / n_prots:.1f}")
    
    print("-" * 40)
    print(f"🔹 PHÂN PHỐI ĐIỂM SỐ (SCORE):")
    print(df['score'].describe().to_string())
    print(f"\n   -> Min Score check: {df['score'].min():.4f} (Phải >= 0.85 nếu code đúng)")
    
    print("-" * 40)
    print("🔹 PHÂN BỐ ĐỘ HIẾM (IA) THEO KHOẢNG:")
    
    # Định nghĩa các bins IA
    bins = [-1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1e9]
    labels = [
        "IA == 0",
        "0 → 1",
        "1 → 2",
        "2 → 3",
        "3 → 4",
        "4 → 5",
        "5 → 6",
        "6 → 7",
        "7 → 8",
        "8 → 9",
        "9 → 10",
        "> 10"
    ]
    
    df['ia_bin'] = pd.cut(df['ia'], bins=bins, labels=labels)
    
    ia_dist = df['ia_bin'].value_counts().sort_index()

    for k in labels:
        count = ia_dist.get(k, 0)
        ratio = count / n_rows * 100
        print(f"   {k:8s} : {count:10,}  |  {ratio:6.2f} %")
    
    # =========================================================================
    # 4. ĐÁNH GIÁ (VERDICT)
    # =========================================================================
    
    print("-" * 40)
    print("🚀 ĐÁNH GIÁ KHẢ NĂNG ENSEMBLE:")
    
    # Tiêu chí 1: Min Score
    if df['score'].min() < 0.84:
        print("❌ CẢNH BÁO: Có điểm số thấp (< 0.85). Code lọc chưa chuẩn!")
    else:
        print("✅ Score Quality: Tốt (Toàn bộ là Homology mạnh).")
    
    # Tính Tail theo IA >= 5
    tail = df[df['ia'] >= 5]
    super_rare = df[df['ia'] >= 10]
    
    tail_ratio = len(tail) / n_rows
    super_ratio = len(super_rare) / n_rows
    
    print(f"   - TAIL (IA ≥ 5)      : {len(tail):,}  |  {tail_ratio:.2%}")
    print(f"   - SUPER RARE (IA ≥10): {len(super_rare):,}  |  {super_ratio:.2%}")
    
    if tail_ratio > 0.2:
        print("✅ Rarity: Tốt. KNN tập trung đúng vùng nhãn hiếm.")
    else:
        print("⚠️ Rarity: Thấp. KNN vẫn thiên nhiều về head.")
    
    # Đánh giá Coverage
    if n_prots < 500:
        print("⚠️ Coverage: Rất thấp (< 500 proteins). Ngưỡng đang quá gắt.")
    elif n_prots > 50000:
        print("⚠️ Coverage: Rất cao. KNN đang phủ quá rộng.")
    else:
        print(f"✅ Coverage: Hợp lý ({n_prots:,} proteins có hàng xóm xịn).")

if __name__ == "__main__":
    analyze_knn_candidates()

📊 ANALYZING knn_homology_candidates.tsv...

   Loading IA weights...
----------------------------------------
🔹 TỔNG QUAN:
   - Tổng số dòng (Predictions): 2,006,071
   - Số Protein được 'Cứu' (Covered): 223,968
   - Số Nhãn GO xuất hiện: 14,504
   - Trung bình số nhãn/protein: 9.0
----------------------------------------
🔹 PHÂN PHỐI ĐIỂM SỐ (SCORE):
count    2.006071e+06
mean     9.845593e-01
std      1.455886e-02
min      8.505000e-01
25%      9.797000e-01
50%      9.884000e-01
75%      9.942000e-01
max      1.000000e+00

   -> Min Score check: 0.8505 (Phải >= 0.85 nếu code đúng)
----------------------------------------
🔹 PHÂN BỐ ĐỘ HIẾM (IA) THEO KHOẢNG:
   IA == 0  :    198,370  |    9.89 %
   0 → 1    :  1,014,524  |   50.57 %
   1 → 2    :    341,029  |   17.00 %
   2 → 3    :    167,992  |    8.37 %
   3 → 4    :    117,108  |    5.84 %
   4 → 5    :     54,426  |    2.71 %
   5 → 6    :     45,386  |    2.26 %
   6 → 7    :     21,273  |    1.06 %
   7 → 8    :     21,836  |   

In [5]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict
from tqdm import tqdm

# =============================================================================
# 1. CẤU HÌNH ĐƯỜNG DẪN & THAM SỐ
# =============================================================================
CONFIG = {
    # File KNN Homology Candidate của bạn
    'KNN_FILE': "knn_homology_candidates.tsv",
    
    # File Vocab của hai mô hình chính
    'VOCAB_C95_PATH': "/kaggle/input/c95-cafa6/vocab_C95_remove.csv", # Giả định C95 là dùng toàn bộ nhãn
    'VOCAB_C99_PATH': "/kaggle/input/c99-cafa6/vocab_C99_remove.csv", # C99 Vocab của bạn
    
    # File IA
    'IA_FILE_PATH': "/kaggle/input/cafa-6-protein-function-prediction/IA.tsv", 
}

# =============================================================================
# 2. HÀM XỬ LÝ DỮ LIỆU
# =============================================================================

def load_terms_set(path, term_col='term'):
    """Tải file Vocab hoặc Term list và trả về một Set chứa các GO Terms."""
    try:
        if path.endswith('.tsv') or path.endswith('.txt'):
            # Xử lý trường hợp C95 dùng file train_terms gốc
            df = pd.read_csv(path, sep='\t', usecols=['term'])
        elif path.endswith('.csv'):
            df = pd.read_csv(path, usecols=[term_col])
        else:
            print(f"Warning: Unknown file type for {path}")
            return set()
        
        # Xóa các giá trị trùng lặp và nan
        return set(df[term_col].dropna().unique())
        
    except FileNotFoundError:
        print(f"❌ ERROR: Không tìm thấy file Vocabulary tại {path}")
        return set()
    except Exception as e:
        print(f"❌ ERROR: Lỗi khi tải file {path}: {e}")
        return set()

def load_ia_map(path):
    """Tải file IA và trả về dictionary {term: IA_value}."""
    ia_map = {}
    try:
        # File IA thường là TSV (GO_ID \t IA_value)
        with open(path, 'r') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    try:
                        ia_map[parts[0]] = float(parts[1])
                    except ValueError:
                        continue # Bỏ qua header hoặc dòng lỗi
    except FileNotFoundError:
        print(f"❌ ERROR: Không tìm thấy file IA tại {path}")
    return ia_map

def get_ia_range(ia_value):
    """Phân loại IA vào các khoảng đã định."""
    if ia_value < 4: return '0_to_4_HEAD' # Không quan tâm
    if ia_value < 5: return '4_to_5'
    if ia_value < 6: return '5_to_6'
    if ia_value < 7: return '6_to_7'
    if ia_value < 8: return '7_to_8'
    if ia_value < 9: return '8_to_9'
    if ia_value < 10: return '9_to_10'
    return '>10_SUPER_TAIL'

# =============================================================================
# 3. PHÂN TÍCH ĐÓNG GÓP ĐỘC QUYỀN CỦA KNN
# =============================================================================

def analyze_unique_contribution():
    print("🚀 BẮT ĐẦU PHÂN TÍCH ĐÓNG GÓP ĐỘC QUYỀN CỦA KNN...")
    
    # --- BƯỚC 1: Tải Dữ liệu ---
    set_c95 = load_terms_set(CONFIG['VOCAB_C95_PATH'], term_col='term')
    set_c99 = load_terms_set(CONFIG['VOCAB_C99_PATH'], term_col='term')
    ia_map = load_ia_map(CONFIG['IA_FILE_PATH'])
    
    try:
        # Load KNN file, GIỮ CẢ SCORE để lọc theo ngưỡng 0.90
        knn_df = pd.read_csv(
            CONFIG['KNN_FILE'],
            sep='\t',
            names=['pid', 'term', 'score']
        )
    except FileNotFoundError:
        print(f"❌ ERROR: Không tìm thấy file KNN Candidates tại {CONFIG['KNN_FILE']}")
        return
    
    # ======================= 🔥 LỌC THEO NGƯỠNG 0.90 🔥 =======================
    knn_df = knn_df[knn_df['score'] >= 0.90]
    
    print(f"   -> Sau khi lọc score >= 0.90:")
    print(f"      - Số dòng còn lại: {len(knn_df):,}")
    print(f"      - Số protein còn lại: {knn_df['pid'].nunique():,}")
    
    # Lấy tập hợp các nhãn duy nhất mà KNN tìm thấy (sau lọc)
    set_knn = set(knn_df['term'].unique())
    print(f"      - Số nhãn GO duy nhất còn lại: {len(set_knn):,}")

    # --- BƯỚC 2: Khởi tạo Bộ đếm ---
    # Bộ đếm sẽ đếm số lượng nhãn (độc lập với số lần dự đoán)
    # Khóa: Khoảng IA (ví dụ: '5_to_6')
    # Giá trị: List các GO term thuộc loại đó
    
    unique_counts = {
        'C95_Missing': defaultdict(list),  # Nhãn không có trong C95
        'C99_Missing': defaultdict(list),  # Nhãn không có trong C99
    }
    
    # --- BƯỚC 3: Loop và Phân loại ---
    
    # Chỉ duyệt qua các nhãn mà KNN tìm thấy
    for term in tqdm(set_knn, desc="Phân loại nhãn"):
        ia_value = ia_map.get(term, 0.0) # Nếu không có IA thì mặc định là 0.0
        ia_range = get_ia_range(ia_value)
        
        # 1. Kiểm tra Nhãn Mất (Missing) khỏi C95
        if term not in set_c95:
            unique_counts['C95_Missing'][ia_range].append(term)
            
        # 2. Kiểm tra Nhãn Mất (Missing) khỏi C99
        if term not in set_c99:
            unique_counts['C99_Missing'][ia_range].append(term)


    # --- BƯỚC 4: Báo cáo kết quả ---
    
    print("\n" + "=" * 60)
    print("  🏆 ĐÓNG GÓP ĐỘC QUYỀN CỦA KNN (THEO ĐỘ HIẾM IA)")
    print("=" * 60)
    
    print("\n--- A. NHÃN KHÔNG CÓ TRONG C95 (C95 Missing) ---")
    
    headers = ["Khoảng IA", "Số nhãn ĐỘC QUYỀN", "Tỷ lệ (%)"]
    data_c95 = []
    
    total_c95_missing = sum(len(v) for k, v in unique_counts['C95_Missing'].items() if k != '0_to_4_HEAD')

    for ia_range in ['4_to_5', '5_to_6', '6_to_7', '7_to_8', '8_to_9', '9_to_10', '>10_SUPER_TAIL']:
        count = len(unique_counts['C95_Missing'][ia_range])
        percent = (count / total_c95_missing) * 100 if total_c95_missing else 0
        data_c95.append([ia_range, f"{count:,}", f"{percent:.2f}%"])

    print(pd.DataFrame(data_c95, columns=headers).to_markdown(index=False))
    print(f"\n   -> Tổng số nhãn hiếm (IA >= 4) mà C95 bỏ qua: {total_c95_missing:,}\n")


    print("--- B. NHÃN KHÔNG CÓ TRONG C99 (C99 Missing) ---")

    headers = ["Khoảng IA", "Số nhãn ĐỘC QUYỀN", "Tỷ lệ (%)"]
    data_c99 = []
    
    total_c99_missing = sum(len(v) for k, v in unique_counts['C99_Missing'].items() if k != '0_to_4_HEAD')

    for ia_range in ['4_to_5', '5_to_6', '6_to_7', '7_to_8', '8_to_9', '9_to_10', '>10_SUPER_TAIL']:
        count = len(unique_counts['C99_Missing'][ia_range])
        percent = (count / total_c99_missing) * 100 if total_c99_missing else 0
        data_c99.append([ia_range, f"{count:,}", f"{percent:.2f}%"])

    print(pd.DataFrame(data_c99, columns=headers).to_markdown(index=False))
    print(f"\n   -> Tổng số nhãn hiếm (IA >= 4) mà C99 bỏ qua: {total_c99_missing:,}\n")

if __name__ == "__main__":
    analyze_unique_contribution()

🚀 BẮT ĐẦU PHÂN TÍCH ĐÓNG GÓP ĐỘC QUYỀN CỦA KNN...
   -> Sau khi lọc score >= 0.90:
      - Số dòng còn lại: 2,001,441
      - Số protein còn lại: 223,835
      - Số nhãn GO duy nhất còn lại: 14,504


Phân loại nhãn: 100%|██████████| 14504/14504 [00:00<00:00, 847780.50it/s]


  🏆 ĐÓNG GÓP ĐỘC QUYỀN CỦA KNN (THEO ĐỘ HIẾM IA)

--- A. NHÃN KHÔNG CÓ TRONG C95 (C95 Missing) ---
| Khoảng IA      |   Số nhãn ĐỘC QUYỀN | Tỷ lệ (%)   |
|:---------------|--------------------:|:------------|
| 4_to_5         |                 671 | 26.82%      |
| 5_to_6         |                 536 | 21.42%      |
| 6_to_7         |                 447 | 17.87%      |
| 7_to_8         |                 346 | 13.83%      |
| 8_to_9         |                 221 | 8.83%       |
| 9_to_10        |                 112 | 4.48%       |
| >10_SUPER_TAIL |                 169 | 6.75%       |

   -> Tổng số nhãn hiếm (IA >= 4) mà C95 bỏ qua: 2,502

--- B. NHÃN KHÔNG CÓ TRONG C99 (C99 Missing) ---
| Khoảng IA      |   Số nhãn ĐỘC QUYỀN | Tỷ lệ (%)   |
|:---------------|--------------------:|:------------|
| 4_to_5         |                 369 | 26.51%      |
| 5_to_6         |                 299 | 21.48%      |
| 6_to_7         |                 229 | 16.45%      |
| 7_to_8         |      